# SIIIM-COVID Stratified sampling

1. Split training data into Stratified 5 folds by human label (4) and OPACITIES PER IMAGE/PATIENT (1 2 3 4 5 8)
2. Resize , scale and nomalize dcm files
3. create mask
4. find duplicate 

# Reffernces 
1. EDA https://www.kaggle.com/dschettler8845/visual-in-depth-eda-siim-covid19-detection
2. incorrect anotation ? https://www.kaggle.com/dschettler8845/covid-detection-studies-with-multiple-images-viz
3. duplication count https://www.kaggle.com/kwk100/siim-covid-19-duplicate-training-images?select=pairs.csv
4. convert 2 png https://www.kaggle.com/xhlulu/siim-covid-19-convert-to-jpg-256px/notebook?scriptVersionId=63196459&select=meta.csv

# Problem explain


- NEGATIVE FOR PNEUMONIA

    No lung opacities

- TYPICAL APPEARANCE

    Multifocal bilateral, peripheral opacities with rounded morphology, lower lung–predominant distribution

- INDETERMINATE APPEARANCE

    Absence of typical findings AND unilateral, central or upper lung predominant distribution

- ATYPICAL APPEARANCE

    Pneumothorax, pleural effusion, pulmonary edema, lobar consolidation, solitary lung nodule or mass, diffuse tiny nodules, cavity
    
  
#  Suspect

1. Maybe "INDETERMINATE APPEARANCE" very simillar to "ATYPICAL APPEARANCE" or "NEGATIVE FOR PNEUMONIA"
2. some images has no bouding box but has human label non-neg
3. how to Deal with duplicate label ？？？https://www.kaggle.com/c/siim-covid19-detection/discussion/246597

In [ ]:
!conda install gdcm -c conda-forge -y

In [ ]:
import os
import ast

import math

import pandas as pd
import numpy as np

from tqdm.auto import tqdm

from sklearn.model_selection import StratifiedKFold

import cv2
from PIL import Image
import matplotlib.pyplot as plt


# read dicom
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import csv

In [ ]:
study_df = pd.read_csv("/kaggle/input/siim-covid19-detection/train_study_level.csv")
image_df = pd.read_csv("/kaggle/input/siim-covid19-detection/train_image_level.csv")

In [ ]:
study_df.head()

In [ ]:
_study_total = len(study_df)
print("total data length" , _study_total)

# Stduy data for stratify
_study_neg_row = study_df.loc[study_df["Negative for Pneumonia"] == 1]
_study_tp_row = study_df.loc[study_df["Typical Appearance"] == 1]
_study_nd_row = study_df.loc[study_df["Indeterminate Appearance"] == 1]
_study_ap_row = study_df.loc[study_df["Atypical Appearance"] == 1]

print("Negative for Pneumonia count" , len(_study_neg_row))
print("Typical Appearance count" , len(_study_tp_row))
print("Indeterminate Appearance	count" , len(_study_nd_row))
print("Atypical Appearance count" , len(_study_ap_row))

print("one hot embedding ?", len(_study_neg_row) + len(_study_tp_row) + len(_study_nd_row) + len(_study_ap_row) == _study_total)

In [ ]:
image_df.head()

In [ ]:
# Image data for stratify
_img_nan_row = image_df.loc[image_df["label"] == "none 1 0 0 1 1"]
print("image total count", len(image_df))
print("image none row count", len(_img_nan_row))

In [ ]:
# "../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm"
_one = image_df.loc[image_df["StudyInstanceUID"] == "a7335b2f9815"]
_one

In [ ]:
!ls ../input/siim-covid19-detection/train/a7335b2f9815

In [ ]:
# "../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm"
_one = image_df.loc[image_df["StudyInstanceUID"] == "00086460a852"]
_one

In [ ]:
study_df.head()

# Create Fold

In [ ]:
colnames = {
    "Negative for Pneumonia": 0,
    "Typical Appearance": 1,
    "Indeterminate Appearance": 2,
    "Atypical Appearance": 3
}

In [ ]:
for index, row in study_df.iterrows():    
    for x in colnames:
        if row[x] == 1:
            study_df.loc[index, "bin"] = colnames[x]

In [ ]:
study_df.head()

In [ ]:
for index, row in image_df.iterrows():       
    image_df.loc[index, "bin"] = study_df.loc[study_df["id"] == row["StudyInstanceUID"] + "_study"]["bin"].item()

In [ ]:
image_df.head()

In [ ]:
study_df.loc[study_df["id"] == "ff0879eb20ed_study"]

In [ ]:
fold = 0

X = image_df
y = image_df['bin']
skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)
skf.get_n_splits(X, y)
for train_index, test_index in skf.split(X, y):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    #X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    #y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    image_df.loc[test_index, 'fold'] = int(fold)
    fold +=1 

In [ ]:
image_df

In [ ]:
study_df.loc[study_df["id"] == "5776db0cec75_study"]

In [ ]:
image_df.to_csv("./updated_iamge_level.csv")

# Find duplicate 

In [ ]:
# hengk's duplicate list, we shall verify it 
duplicateList = [
'87adda1908f9','35e398a5a431','1b92142f4362','ef6e312ca719','866e3622cb24','3577ee4f26c4','d74ef8961bff','6f54e9cbd180','b97c6b32105e','8093df07a5d0','efc93a3917b6','b3ffe59e37c7','9939f63af4ff','00c1515729a8','267a250932bc','3e7b2ffc97db','b121806162c3','2f973757eaa4','a39667fe9a81','a5bbd30ed109','07c6b234a94b','089bd77c8c10','0575769456b9','149356f04849','ace7a9702770','0acec7010081','611348a721f7','deea6d6f81a5','b0866caa201a','ab55abb953d1','7626c521cad7','bee62c601ae9','582c442e440b','869476b0763a','9b1de1c45491','32222cc776a2','66dabc6f972d','1dc7459cb081','a2ee4b862182','3b6ad60071d4','39d52f244db3','ce51b397b1a6','2a7a456d367e','a94171e98807','21518ca15050','e96133d06736','ea2688741043','7b7275d670e7','59bc532be971','684230477525','8d4b3609ed92','608d574388ba','76c66ee8e58d','6ff218d10741','7966f780f27f','7f1924880cf8','0102b5cac730','99f3642f50f5','6e9fad584bff','830063223a31','72cf260ddf4c','156cb1f5c689','6d36ffbc7864','9fcbe25a88e0','6534a837497d','04cc2f7f4c4b','30b18db28900','1c1069c57757','f208dc529d16','df2bb22fa871','05c063f5cef5','eea3a910fa9e','2c130ee08736','c6e92e59a0ae','1ea01196514a','4c414b793562','00e3a7e91a34','6e5946091b8a','c3a09e8a600d','9872a8a48f23','12e97ed89297','66712e2fc6a4','c843d08b49b8','df4f1240317e','a5a364383f34','0dd32589bdc3','93a881fb3292','550f057ee0b0','ea2117b53323','6728e11290af','be65d1a22de5','08acae0bf785','0d4d6acc9ed3','779f0040d1b2','caa7fd25ee9d','e73f2cc047ad','75b52bec817f','ddb051c1233b','9108cdfd43dc','a57bf6dd6119','c4b68b29a072','717ea5155b46','49664f078f0e','e738c549fe8e','4cbc17936e7d','e3e2f20e0264','b5a415f70aa9'
]

In [ ]:
len(duplicateList)

In [ ]:
duplicate_df_path = '../input/siim-covid-19-duplicate-training-images/duplicates.csv'

In [ ]:
_result = []
with open(duplicate_df_path, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        rows = row[0].split(',')
        _count = 0
        _cand = []
        for item in rows:
            _org = image_df.loc[image_df['id'] == item]
            if str(_org['boxes'].item()) != 'nan':
                _count +=1
            else:
                _cand.append(item)
        if _count == 0:
            _result += _cand[1:]
        if _count > 0: 
            _result += _cand
        if _count > 1:
            print(row)

In [ ]:
_result = [x.replace('_image','') for x in _result]
#_result
len(_result)

In [ ]:
# duplicate wrong anotation
_result.append('7b7275d670e7')

In [ ]:
count = 0
for x in _result:
    if x in duplicateList:
        count += 1
print(count)

In [ ]:
with open('dublicate.txt', 'w') as text_file:    
    # using csv.writer method from CSV package
    text_file.write('\n'.join(_result))

# Create PNG files

In [ ]:
_SIZE_ = 512 
_Type_ = "png"

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
path = '../input/siim-covid19-detection/train/ae3e63d94c13/288554eb6182/e00f9fe0cce5.dcm'
dicom = pydicom.read_file(path)

In [ ]:
dicom

In [ ]:
example = read_xray(path)

In [ ]:
plt.imshow(example, cmap="bone")

# Create mask

In [ ]:
example

In [ ]:
image_df

In [ ]:
!ls ../input/siim-covid19-detection/train/e28208b7c58e/4b7cd369719c

In [ ]:
path2 = '../input/siim-covid19-detection/train/e28208b7c58e/4b7cd369719c/0a0bb7af0cab.dcm'
dicom2 = pydicom.read_file(path)

In [ ]:
example2_meta = image_df.loc[image_df['id'] == "0a0bb7af0cab_image"]
example2_meta

In [ ]:
image_df

In [ ]:
bbox = example2_meta['boxes'].item()
bbox

In [ ]:
example2 = read_xray(path2)

In [ ]:
example2.shape

In [ ]:
for box in ast.literal_eval(bbox):
    x1, y1, x2, y2 = box["x"], box["y"], box["x"] + box["width"], box["y"] + box["height"]
    x1 = int(x1) 
    y1 = int(y1) 
    x2 = int(x2)  
    y2 = int(y2) 
    cv2.rectangle(example2, (x1, y1), (x2, y2), (0, 80, 255), 15)

In [ ]:
plt.imshow(example2, cmap="bone")

In [ ]:
blankimg = np.zeros((example2.shape[0],example2.shape[1],3), dtype=np.uint8)
for box in ast.literal_eval(bbox):
    x1, y1, x2, y2 = box["x"], box["y"], box["x"] + box["width"], box["y"] + box["height"]
    x1 = int(x1) 
    y1 = int(y1) 
    x2 = int(x2)  
    y2 = int(y2) 
    cv2.rectangle(blankimg, (x1, y1), (x2, y2), (255, 255, 255), -1)

In [ ]:
plt.imshow(blankimg, cmap="bone")

In [ ]:
image_id = []
dim0 = []
dim1 = []
splits = []
study = []

maskdir = "/kaggle/tmp/mask/"
os.makedirs(maskdir, exist_ok=True)
    
#for split in ['test', 'train']:
for split in ['train', 'test']:
    save_dir = f'/kaggle/tmp/{split}/'
    
    os.makedirs(save_dir, exist_ok=True)
    
    for study_id in os.listdir(f'../input/siim-covid19-detection/{split}'):
        for dirname, _, filenames in os.walk(f'../input/siim-covid19-detection/{split}/{study_id}'):
            for file in filenames:
                # set keep_ratio=True to have original aspect ratio
                xray = read_xray(os.path.join(dirname, file))

                im = resize(xray, size=_SIZE_)
                _name = file.replace('dcm', _Type_)
                im.save(os.path.join(save_dir, _name))

                _img_id = file.replace('.dcm', '')
                image_id.append(_img_id)

                _w = xray.shape[0]
                _h = xray.shape[1]
                dim0.append(_w)
                dim1.append(_h)
                splits.append(split)
                study.append(study_id)

                if split != 'test':
                    _mate = image_df.loc[image_df['id'] == _img_id + "_image"]
                    _bbox = _mate['boxes'].item()
                    if str(_bbox) != "nan":
                        blankimg = np.zeros((_w, _h,3), dtype=np.uint8)

                        for box in ast.literal_eval(_bbox):
                            x1, y1, x2, y2 = box["x"], box["y"], box["x"] + box["width"], box["y"] + box["height"]
                            x1 = int(x1) 
                            y1 = int(y1) 
                            x2 = int(x2)  
                            y2 = int(y2) 
                            cv2.rectangle(blankimg, (x1, y1), (x2, y2), (255, 255, 255), -1)

                        blankimg = resize(blankimg, size=_SIZE_)
                        blankimg.save(os.path.join(maskdir, _name))

In [ ]:
%%time
!tar -zcf train.tar.gz -C "/kaggle/tmp/train/" .
!tar -zcf test.tar.gz -C "/kaggle/tmp/test/" .
!tar -zcf mask.tar.gz -C "/kaggle/tmp/mask/" .

In [ ]:
meta_df = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits, 'study': study})
meta_df.to_csv('meta.csv', index=False)